# Foundations of Linear Regression

## Exercise 1: Understanding the Likelihood of Observations Under a Given Gaussian Model 

### Question 1a: Gaussian Likelihood Function
Write a function to compute the likelihood of observing a set of values under a normal distribution with mean mu and variance sigma^2. The function should accept an observation Y, a mean value mu, and a standard deviation sigma, and return the likelihood of observing Y under these parameters.

In [37]:
import math

# The likelihood of y given mu and sigma
def normal_pdf(y, mu, sigma):
    return (1 / math.sqrt(2 * math.pi * (sigma**2))) * math.exp((-1 / 2 * (sigma**2)) * ((y - mu)**2))

0.3989422804014327

### Question 1b: Likelihood Under Standard Normal Model
Simulate a sequence of values from -5 to 5 (with a step of 0.1) and calculate the likelihood of each value under a Standard Normal model (mu=0, sigma=1). Plot these likelihood values and discuss the shape of the plot.

In [ ]:
import matplotlib.pyplot as plt

## Exercise 2: Playing with Linear Regression

We will work with the `Time-for-change` regression model to predict the results of the US elections. The goal of the model is to estimate the % of the two-party (Republican + Democrat) vote from 3 variables: the `incumbency` (i.e. whether the incumbent party candidate is seeking re-election or if it's a new candidate) of the president; the growth rate of the economy; and the approval rating of the incumbent party president. 

### Question 2a: Linear Regression Model
Estimate a linear regression model to predict vote share using at least one categorical variable (if applicable) and one numeric variable.

In [ ]:
import os

# Change working directory to where you store the Camp files 
os.chdir('/Users/Sari/Documents/Code/Semester 4 workbooks')

In [ ]:
import pandas as pd

# load Historical Election Results dataset
df = pd.read_csv('Camp_Election.Oracle/Historical Election Results/MIT_Lab_Historical_Election_Results/1976-2020-president.csv')

Here I want to include in this dataset the necessary tools to perform the `time for change` model. I need to augment the dataset with a) the `incumbency` of the party / president ; b) the real GDP growth in the year of the election; c) the `approval rating` of the incumbent president on the eve of the election. 

In [ ]:
# Here we do a little data Wrangling to improve the dataset for analysis

# Focus on Democrats and Republicans only 
df = df[df['party_simplified'].str.contains('DEMOCRAT') | df['party_simplified'].str.contains('REPUBLICAN')]

# Calculate the two-party total votes for each state and year
df['two_party_total'] = df.groupby(['year', 'state'])['candidatevotes'].transform('sum')

# Calculate the percentage of the two-party vote afforded to each party
df['pct_2p_vote'] = 100*df['candidatevotes'] / df['two_party_total']

# define a function that returns the incumbent and whether the incumbent party is seeking their 2nd term in office
def identify_incumbent_complete(year):
    if year in [1976]:
        return ('REPUBLICAN', 0)  # Ford (Continuing Nixon's term)
    elif year in [1980]:
        return ('DEMOCRAT', 1)  # Carter
    elif year in [1984, 1988]:
        return ('REPUBLICAN', 1 if year == 1984 else 0)  # Reagan
    elif year in [1992]:
        return ('REPUBLICAN', 0)  # H.W. Bush (continuing Republican (Reagan) term)
    elif year in [1996, 2000]:
        return ('DEMOCRAT', 1 if year == 1996 else 0)  # Clinton
    elif year in [2004, 2008]:
        return ('REPUBLICAN', 1 if year == 2004 else 0)  # W. Bush
    elif year in [2012, 2016]:
        return ('DEMOCRAT', 1 if year == 2012 else 0)  # Obama
    elif year in [2020]:
        return ('REPUBLICAN', 1)  # Trump
    else:
        return ('None', 0)  # Default for years without a U.S. presidential election

# Apply the function to each row
df['incumbent_party'] = 'None'
df['term2'] = 0
for index, row in df.iterrows():
    incumbent_party, term2 = identify_incumbent_complete(row['year'])
    df.at[index, 'incumbent_party'] = incumbent_party
    df.at[index, 'term2'] = term2

# Now we can focus on the incumbent party, and focus our analysis to ask `what is the chance that the share of the vote the incmumbent will get ? 
df['incumbent_dummy'] = np.where(df['incumbent_party']==df['party_simplified'],1,0)
df = df[df['incumbent_dummy'] == 1]

# and we only need a few variables 
df = df[['year','state','pct_2p_vote','term2']]

In [ ]:
# augment with national GDP data 
aux_gdp = pd.read_csv('Camp_Election.Oracle/Historical_National.US.GDP.csv')
aux_gdp = aux_gdp[aux_gdp['Items']=='Gross domestic product']

# melt 
aux_gdp = pd.melt(aux_gdp, id_vars='Items',var_name='year',value_name='real_gdp_pct_growth')
aux_gdp = aux_gdp[['year','real_gdp_pct_growth']]
aux_gdp['real_gdp_pct_growth'] = pd.to_numeric(aux_gdp['real_gdp_pct_growth'])

# merge with df 
df['year'] = df['year'].astype(int)  # Convert to int
aux_gdp['year'] = aux_gdp['year'].astype(int)  # Ensure this matches df
df = pd.merge(df, aux_gdp, on='year', how='left')

In [ ]:
# augment with incumbent's approval rating
aux_app = pd.read_csv('Camp_Election.Oracle/Historical_President.Approval.Polls_Gallup.csv')
aux_app['net_approval'] = aux_app['Approving'] - aux_app['Disapproving']

# select only the approval polls right closest to the election
import pandas as pd

def last_approval_before_election(data):
    """
    Selects the last approval poll before each presidential election in the dataset.
    
    :param data: A pandas DataFrame with columns 'Start Date', 'End Date', 'Approving', 'Disapproving', 'Unsure/NoData', and 'net_approval'.
    :return: A pandas DataFrame with the selected polls.
    """
    # Convert date columns to datetime
    data['Start Date'] = pd.to_datetime(data['Start Date'])
    data['End Date'] = pd.to_datetime(data['End Date'])
    
    # Sort the data by 'End Date' to ensure chronological order
    data.sort_values(by='End Date', inplace=True)
    
    # Identify the range of years in the dataset
    start_year = data['End Date'].dt.year.min()
    end_year = data['End Date'].dt.year.max()
    
    # Presidential elections are held every 4 years, starting from 1944
    election_years = [year for year in range(start_year, end_year+1) if year % 4 == 0 and year >= 1944]
    
    selected_polls = []
    for year in election_years:
        # Election date: first Tuesday after the first Monday in November
        # Simplification: use November 1st as a base, then adjust
        base_date = pd.Timestamp(year=year, month=11, day=1)
        day_of_week = base_date.dayofweek
        election_date = base_date + pd.Timedelta(days=(1-day_of_week) % 7 + 1)
        
        # Select the last poll before the election
        last_poll = data[data['End Date'] < election_date].iloc[-1]
        selected_polls.append(last_poll)
    
    return pd.DataFrame(selected_polls)

# apply the function
aux_app = last_approval_before_election(aux_app)

# extract relevant year
aux_app['year'] = aux_app['End Date'].astype(str).str[:4].astype(int)

# drop unnecessary variables 
aux_app = aux_app[['year','net_approval']]

# merge with election results dataset 
df = pd.merge(df, aux_app, on='year', how='left')

In [ ]:
# Let's see the results of our effors
print(df)

# Write this to a csv so we can use it in later workshops
df.to_csv('Workshop_I__Foundations_of_Linear_Regression/generated_quantities/clean_TFC_data.csv', index=False)

### Now we estimate the `time for change` regression

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Define your design matrix and your target variable 
...

# Add a constant to the features to account for the intercept in the model
...

# Fit the model
...
print(model.summary())

### b) Inference by `Empirical Posterior` Simulations

Simulate 1000 values from the empirical posterior distribution of each of your coefficients. Plot the histogram for each of these coefficient simulations, and calculate: i. the proportion of simulation that are above 0; ii. the Monte Carlo Median and 95% prediction interval for each coefficient; iii. Comment on the significance and magnitude of the effects. 

In [ ]:
import matplotlib.pyplot as plt

# Simulate values from empirical posterior distribution of model coefficients

# Hint: use the np.random.multivariate_normal function to simulate form a multivariate normal, 
# covariance matrix `cov_params()' from the model. 


### c) Generate Predicted Values and Estimate Statistics

For each record in your dataset, generate predicted values. Use the package which you are using for fitting the regression model to output a) point estimates for each fitted value; b) the expected standard error of those estimates. Use these to generate 1000 samples from the empirical posterior distribution of the fitted values, and estimate the Monte Carlo Median and the 95% prediction interval.

In [ ]:
import statsmodels.api as sm


# Step 1: Prepare the new dataset - remember to add a constant if your model includes an intercept
X_new = X
...

# Step 2: Predict new values (use the model.predict function)
...

# Step 3: Calculate standard errors of the predictions
cov_matrix = ...
design_matrix = X_new_with_const
variances = np.diag(np.dot(np.dot(design_matrix, cov_matrix), design_matrix.T))
std_errors_predictions = ...

# Generate posterior samples
new_predictions_array = np.array(new_predictions)[:, None]  # Convert to numpy array and add new axis
std_errors_predictions_array = np.array(std_errors_predictions)[:, None]  # Convert to numpy array and add new axis

predicted_samples = np.random.normal(loc=...,
                                     scale=...,
                                     size=(..., 1000))

monte_carlo_medians = ...
prediction_intervals = np.percentile(...)


### d) Plot Observed vs. Predicted Values with Prediction Intervals

Plot the observed values (on the y-axis) against the Monte Carlo Median of the fitted values (x-axis) for each observation. On the same plot, find a way to add the prediction intervals around your fitted values. Make sure the y and x axis span the same support, and add a y = x line to the plot. Comment on what the plot shows - does your model recover good predictions ? How can you tell ? Play around with your model and look how including / removing complexity changes the performance. 

In [ ]:
...

# Plotting
scatter = plt.scatter(..., y,...)
errorbar = plt.errorbar(..., y, xerr=np.abs(prediction_intervals - monte_carlo_medians), ...)
y_equals_x = plt.plot(..., 'k--', zorder=3, label='y = x')

...

plt.title('Observed vs. Predicted Values')
plt.xlabel('Predicted Values')
plt.ylabel('Observed Values')
plt.show()
